# SkyWalking实践

## 参考文献

* *https://apisix.apache.org/zh/docs/apisix/plugins/skywalking/*
* *https://github.com/hotbaby/apisix-conf*
* *https://skywalking.apache.org*

## 搭建SkyWalking

|组件|版本|服务|
|--|----|----|
|APISIX|`2.6`|APISIX|
|elasticsearch|`6.7.2`|存储后端|
|skywalking|`8.3.0-es6`|skywalking服务|
|skywalking-ui|`8.3.0`| skywalking UI|


> **重要： elasticsearch、skywalking和skywalking-ui的版本一定匹配，否则可能启动失败，APISIX agent无法接入。**

### 启动Elasticsearch

```shell
docker kill elasticsearch
docker rm elasticsearch
docker run -d --restart always \
              --name elasticsearch \
              -p 9200:9200 \
              -p 9300:9300 \
              -e "discovery.type=single-node" \
              -v /data/elasticsearch/data:/usr/share/elasticsearch/data \
              -v /data/elasticsearch/logs:/usr/share/elasticsearch/logs \
              elasticsearch:6.7.2
```

>提前创建` /data/elasticsearch/data`和` /data/elasticsearch/logs`目录

### 启动`SkyWalking`

等待elasticsearch启动成功后，再启动skywalking。

```shell

docker kill skywalking
docker rm skywalking
docker run -d --restart always \
              --name skywalking \
              -p 11800:11800 \
              -p 12800:12800 \
              -e TZ=Asia/Shanghai \
              --link elasticsearch:elasticsearch \
              -e SW_STORAGE=elasticsearch \
              -e SW_STORAGE_ES_CLUSTER_NODES=elasticsearch:9200 \
              apache/skywalking-oap-server:8.3.0-es6
```

### 启动 SkyWalking UI

等待skywalking oap启动成功，再启动skywalking-ui。

```shell
docker kill skywalking-ui
docker rm skywalking-ui
docker run -d --restart always \
              --name skywalking-ui \
              -p 8080:8080 \
              --link skywalking:skywalking \
              -e SW_OAP_ADDRESS=skywalking:12800 \
              apache/skywalking-ui:8.3.0
```

启动成功，可以通过`http://ip:8080`访问skywalking-ui。

## SkyWalking接入

### APISIX接入SkyWalking

配置APISIX

`confg/config.yaml`

```yaml
plugins:                          # plugin list (sorted in alphabetical order)
  - api-breaker
  - skywalking

plugin_attr:
  skywalking:
    service_name: APISIX
    service_instance_name: "APISIX Ubuntu Instance"
    endpoint_addr: http://skywalking:12800


```

配置路由`skywalking`插件。

**不要配置全局`skywalking`插件，否则会报错，参考 [issue](https://github.com/apache/apisix/issues/4571).**

### Python客户端接入SkyWalking

安装`apache-skywalking`Python依赖包：

```shell
pip install apache-skywalking==0.6.0
```

Flask demo `sw_agent.py`

```python
# encoding: utf8

import logging
from flask import Flask
from skywalking import agent, config

config.init(collector='172.30.12.122:11800', service='Flask', instance='MBP')
agent.start()


app = Flask(__name__)


@app.route('/test', methods=['GET'])
def demo():
    return "Test skywalking"


if __name__ == '__main__':
    logging.basicConfig()
    app.run()

```

测试

```shell
for i in $(seq 1 1000); do echo $i && curl http://localhost:5000/test && sleep .1; done
```

## FAQ

略